In [50]:
from functools import reduce
import numpy as np

In [51]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

In [52]:
def swap(array, start, length):
    N = len(array)
    end = (start + length - 1) % N
    for i in range(length//2):
        j = (start + i) % N
        k = (end - i) % N
        array[j], array[k] = array[k], array[j]
    return array

In [53]:
def str_to_255(s):
    return [ord(i) for i in s]

In [54]:
def knot_hash(string):
    seq = str_to_255(string)
    array = list(range(256))
    suffix = [17, 31, 73, 47, 23]
    seq += suffix

    skip = 0 
    pos = 0
    
    for i in range(64):
        for size in seq:
            swap(array, pos, size)
            pos += size + skip
            skip += 1
    
    xors = [reduce(lambda a,b: a^b, array[16*i:16*(i+1)]) for i in range(16)]
    out = "".join(f"{x:08b}" for x in xors)
    return out
    

In [55]:
@memoize
def disk(seed):
    disk = np.zeros((128,128), dtype=int)
    for i in range(128):
        s = f'{seed}-{i}'
        h = knot_hash(s)
        disk[i] = np.array([int(i) for i in h])
    return disk

In [57]:
def count_squares(seed):
    return np.sum(disk(seed))

In [60]:
count_squares('flqrgnkx')

8108

In [61]:
count_squares('ugkiagan')

8292

## P14.2

In [114]:
# def diagonal_generator(N):
#     N -= 1
#     i = 0
#     while True:
#         k = max(0,i-N)
#         if k >= i+1-k:
#             return
#         for j in range(k,i+1-k):
#             yield (j, i-j)
#         i += 1

In [115]:
# list(diagonal_generator(4))

In [109]:
import itertools

In [111]:
def connected_components(graph):
    components = []
    i = 0
    visited = set()
    for node in graph:
        if node in visited:
            continue
        else:
            components.append(set())
            queue = list(graph[node])
            components[-1].add(node)
            while len(queue) > 0:
                node = queue.pop(0)
                visited.add(node)
                queue.extend([c for c in graph[node] if c not in visited])
                if node not in components[-1]:
                    components[-1].add(node)
    return components

In [129]:
np.argwhere(disk('flqrgnkx'))

array([[  0,   0],
       [  0,   1],
       [  0,   3],
       ...,
       [127, 124],
       [127, 125],
       [127, 126]])

In [134]:
def disk_regions(disk):
    graph = {}
    for x,y in np.argwhere(disk):
        graph[(x,y)] = []
        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
            if 0 <= x+dx < 128 and 0 <= y+dy < 128 and disk[x+dx,y+dy] == 1:
                graph[(x,y)].append((x+dx,y+dy))

    cc = connected_components(graph)
    return len(cc)
            

In [135]:
disk_regions(disk('flqrgnkx'))

1242

In [136]:
disk_regions(disk('ugkiagan'))

1069